In [1]:
import urllib.request
import tarfile
# from couchdb_settings import *
# from functions import *


# download file from dropbox & unzip

url = 'https://www.dropbox.com/s/y4ss9gmw6ykde51/twitter-melb.json.tar.gz?dl=1'
filename = 'twitter-melb.tar.gz'

def historic_data(url, filename): 

    urllib.request.urlretrieve(url, filename)

    if filename.endswith('tar.gz'):
        tar = tarfile.open(filename, 'r:gz')
        tar.extractall()
        tar.close()
        
historic_data(url, filename)

In [3]:
# process historic tweets & save to db
import itertools
import json

twitter_path = 'twitter-melb.json'

def read_tweets(twitter_path):

    with open(twitter_path, 'r', encoding = 'utf-8') as f:
        for line in f:
            yield line
                 
def proc_line(line):
    line = line.strip('\r\n')
    line = line.strip('')
    if line.endswith (','):
        line = line[:-1]
    if line.endswith (']}'): 
        line = line[:-2]
    return line

yield_tweets = read_tweets(twitter_path)

# tweet_db = db_connect(tweets)


top5 = itertools.islice(yield_tweets, 5) 

for line in top5:
    line_processed = proc_line(line)
    try:
        data = json.loads(line_processed)
        if data['doc']['place']['id'] == '01864a8a64df9dc4':
            tweet_id = data['id']
            print(tweet_id)
        # if tweet_id not in tweet_db and tweet_id not in test_db:
        #     save_to_db(tweet_id, 'historic', data['doc'], test_db)
    except Exception as e:
        print('Error reading line --ignoring')
        print(e)      

Error reading line --ignoring
Expecting value: line 1 column 51 (char 50)
493802764774825984
493802997793570817
493803077489537024
493803727690559492


In [3]:
################################
# split to save to EC2 CouchDB #
################################

import os
import json
twitter_path = 'twitter-melb.json'
filesize = os.path.getsize(twitter_path)
print(filesize)

chunksize = filesize / 100

10948035094


In [4]:
def chunk(twitter_path, chunksize, filesize):
    
    with open(twitter_path, 'rb') as f:
        # Get current position in file
        chunk_end = f.tell()

        while True:
            chunk_start = chunk_end
            # Move from current position to current position + chunk_size
            f.seek(f.tell() + chunksize)
            # Read until the next line to not break lines in the middle
            f.readline()
            # Get chunk_end as the offset from the beginning of the file in bytes
            chunk_end = f.tell()
            if chunk_end > filesize:
                chunk_end = filesize
            yield chunk_start, chunk_end - chunk_start
            if chunk_end == filesize:
                break

def read_tweets(twitter_path, start, size):
    """
    param: twitter file path
    return: twitter file lines
    """

    with open(twitter_path, 'rb') as f:
        # Get current position in file
        f.seek(start)
        end = start
        while True:
            line = f.readline()
            end = f.tell()
            yield line
            if end > start + size:
                end = start + size
            if end == start + size:
                break
            
def proc_line(line):
    """
    param: twitter file lines
    return: processed twitter file lines
    """
    line = line.decode('utf-8')
    line = line.strip('\r\n')
    line = line.rstrip('')
    if line.endswith (','):
        line = line[:-1]
    if line.endswith (']}'): 
        line = line[:-2]

    return line

In [5]:
# couchbd_settings
address = 'localhost:15984'
username = 'admin'
password = '123456'
tweets = 'raw_tweets'
test = 'view-test'

In [7]:
import couchdb

# use authenticated connection to connect to couchdb server for writing to the database
def db_connect(dbname):

    couchserver = couchdb.Server('http://' + username + ':' + password + '@' + address)
    try:
      db = couchserver[dbname]
    except:
      db = couchserver.create(dbname)

    return db

test_db = db_connect(test)
tweet_db = db_connect(tweets)

def save_to_db(item_id, keyword, item, db):
    """"""
    """"""

    db[item_id] = {keyword: item}

In [8]:
chunks = []

for chunkStart, chunkSize in chunk(twitter_path, int(chunksize), filesize):
            chunks.append({'chunkStart': chunkStart, 'chunkSize': chunkSize})

In [9]:
import itertools

chunkStart = chunks[10]['chunkStart']
chunkSize = chunks[10]['chunkSize']
yield_tweets = read_tweets(twitter_path,  chunkStart, chunkSize)

for line in yield_tweets:
    line_processed = proc_line(line)
    try:
        data = json.loads(line_processed)
        tweet_id = data['id']
        if tweet_id not in tweet_db and tweet_id not in test_db:
            save_to_db(tweet_id, 'historic', data['doc'], test_db)
    except Exception as e:
        print('Error reading line --ignoring')
        print(e)      
        

KeyboardInterrupt: 